# Blaauw Observatory Archive 
### Demo / Tutorial
##### _02/10/2020_
#### Sten Sipma - sipma@astro.rug.nl

## Overview
- Introduction
- SQL & ADQL Basics
- Tap Interfaces
- Python Interface & Example Queries
- Future Development

# Introduction

- Turn folders located at: `/net/vega/data/users/observatory/images/`

- Into:
[http://vo.astro.rug.nl/](http://vo.astro.rug.nl/)

## Blaauw Observatory Archive VO:
A database containing the FITS headers of each observation!

Accessible through:
- Web Interface:  [http://vo.astro.rug.nl/](http://vo.astro.rug.nl/)
- TAP:            [http://vo.astro.rug.nl/tap](http://vo.astro.rug.nl/tap)

# SQL & ADQL basics

SQL (Structured Query Language) is used to define and query those databases. We will restrict ourselves to just querying the database!

In [1]:
# standard python library
import sqlite3

# pip install tabulate
from tabulate import tabulate

In [2]:
data = [("Alice", 20), ("John", 73), (None, 0), ("Bob", 37), ("Eve", None), ("Jane", 91), ("Claire", 42)]
coldata = [("Alice", "Green"), ("Bob", "Magenta"), ("Eve", "White"), ("Claire", "Purple")]

con = sqlite3.connect(":memory:")
with con as curs:
    curs.execute("CREATE TABLE record (name VARCHAR, age INTEGER);")
    curs.executemany("INSERT INTO record VALUES (?, ?)", data)
    curs.execute("CREATE TABLE colours (name VARCHAR, colour VARCHAR);")
    curs.executemany("INSERT INTO colours VALUES (?, ?)", coldata)
    
curs = con.cursor()

In [3]:
def print_table(table, **kwargs):
    if 'headers' not in kwargs:
        kwargs['headers'] = ["Name", "Age"]
    print(tabulate(table, tablefmt='github', **kwargs))

def make_query(query):
    curs.execute(query)
    result = curs.fetchall()
    return result

def print_query(query, **kwargs):
    print_table(make_query(query), **kwargs)

Given some table named `record`, with names and ages. Note that some values are not available (they are `NULL`):

In [4]:
print_table(data)

| Name   |   Age |
|--------|-------|
| Alice  |    20 |
| John   |    73 |
|        |     0 |
| Bob    |    37 |
| Eve    |       |
| Jane   |    91 |
| Claire |    42 |


The basic query looks like:
```
SELECT <columns>
FROM <table>
```

To show everything, we do:
```
SELECT name, age 
FROM record
```

In [5]:
print_query("SELECT name, age FROM record")

| Name   |   Age |
|--------|-------|
| Alice  |    20 |
| John   |    73 |
|        |     0 |
| Bob    |    37 |
| Eve    |       |
| Jane   |    91 |
| Claire |    42 |


Or more conveniently, to select all columns with a wildcard:
```
SELECT *
FROM record
```

In [6]:
print_query("SELECT * FROM record")

| Name   |   Age |
|--------|-------|
| Alice  |    20 |
| John   |    73 |
|        |     0 |
| Bob    |    37 |
| Eve    |       |
| Jane   |    91 |
| Claire |    42 |


You do not have to display all columns:
```
SELECT name
FROM record
```

In [7]:
print_query("SELECT name FROM record")

| Name   |
|--------|
| Alice  |
| John   |
|        |
| Bob    |
| Eve    |
| Jane   |
| Claire |


### Filtering Rows

More usefull queries contain a where clause, which filters based on a condition.
```
SELECT * FROM record
WHERE <condition>
```

For example, if we wanted everyone younger than 30 years, we do:
```
SELECT * FROM record
WHERE age < 30
```

In [8]:
print_query("SELECT * FROM record WHERE age < 30")

| Name   |   Age |
|--------|-------|
| Alice  |    20 |
|        |     0 |


Note, that the `NULL` in age value is automatically evaluated as `False` and is not included.

The condition can be any boolean expression, and therefore can contain `AND`, `OR`, `NOT` etc:

```
SELECT * FROM record
WHERE (age > 30 AND age < 50) OR name == 'Alice'
```

In [9]:
print_query("SELECT * FROM record WHERE (age > 30 AND age < 50) OR name == 'Alice'")

| Name   |   Age |
|--------|-------|
| Alice  |    20 |
| Bob    |    37 |
| Claire |    42 |


We can test if values exist (e.g. are not NULL), by `<column> IS NOT NULL`:
```
SELECT *
FROM record
WHERE name IS NOT NULL 
  AND age  IS NOT NULL
```

In [10]:
print_query("SELECT * FROM record WHERE name IS NOT NULL AND age IS NOT NULL")

| Name   |   Age |
|--------|-------|
| Alice  |    20 |
| John   |    73 |
| Bob    |    37 |
| Jane   |    91 |
| Claire |    42 |


Another usefull tests for strings is `LIKE <regex>` which essentially tests if the value matches a (simple) regular expression. `%` is used to match 0 or more characters, and `.` to match exactly one. 

The following queries every record row, where the name contains an 'a' (case insensitive):

```
SELECT * FROM record
WHERE name LIKE '%a%'
```

In [11]:
print_query("SELECT * FROM record WHERE name LIKE '%a%' ")

| Name   |   Age |
|--------|-------|
| Alice  |    20 |
| Jane   |    91 |
| Claire |    42 |


### Ordering Output
We can also sort the output of our query, using the `ORDER BY` clause:
```
SELECT *
FROM record
ORDER BY age
```

In [12]:
print_query("SELECT * FROM record ORDER BY age")

| Name   |   Age |
|--------|-------|
| Eve    |       |
|        |     0 |
| Alice  |    20 |
| Bob    |    37 |
| Claire |    42 |
| John   |    73 |
| Jane   |    91 |


And this can be specified as ascending (default) or descending (keywords `ASC` and `DESC` respectively)
```
SELECT *
FROM record
ORDER BY name DESC
```

In [13]:
print_query("SELECT * FROM record ORDER BY name DESC")

| Name   |   Age |
|--------|-------|
| John   |    73 |
| Jane   |    91 |
| Eve    |       |
| Claire |    42 |
| Bob    |    37 |
| Alice  |    20 |
|        |     0 |


### Combining tables with JOIN
Finally, if you are working with multiple tables (which is not the case at the moment) we can merge them together using the `JOIN` keyword!

Consider another table (`colours`) with the favourite colours of _some_ people :

In [14]:
print_query("SELECT * FROM colours")

| Name   | Age     |
|--------|---------|
| Alice  | Green   |
| Bob    | Magenta |
| Eve    | White   |
| Claire | Purple  |


We combine entries in the two tables (`<table1> JOIN <table2>`), if their names are the same (`ON <condition>`):

We explicitly state that we display the name from the record table, using `record.name`.
```
SELECT record.name, age, colour
FROM (record JOIN colours 
      ON record.name = colours.name)
```

In [15]:
print_query("SELECT record.name, age, colour FROM record JOIN colours ON record.name = colours.name", headers=["Name", "Age", "Colour"])

| Name   |   Age | Colour   |
|--------|-------|----------|
| Alice  |    20 | Green    |
| Bob    |    37 | Magenta  |
| Eve    |       | White    |
| Claire |    42 | Purple   |


Notice, how names which are in `record` but not in `colours`, are not included. This is called an `INNER` join, and is the default behaviour. 

To include all values and still merge those which match, we do an `OUTER` join:

```
SELECT record.name, age, colour
FROM (record LEFT OUTER JOIN colours 
      ON record.name = colours.name)
```

In [16]:
print_query("SELECT record.name, age, colour FROM record LEFT OUTER JOIN colours ON record.name = colours.name", headers=["Name", "Age", "Colour"])

| Name   |   Age | Colour   |
|--------|-------|----------|
| Alice  |    20 | Green    |
| John   |    73 |          |
|        |     0 |          |
| Bob    |    37 | Magenta  |
| Eve    |       | White    |
| Jane   |    91 |          |
| Claire |    42 | Purple   |


## Now on to ADQL

ADQL (Astronomy Data Query Language) is a separate language build on top of SQL. It offers a lot of constructs and (mathematical) functions which are usefull to astronomers!

For a more comprehensive guide, see the [GAIA DR1 ADQL Cookbook](https://www.gaia.ac.uk/data/gaia-data-release-1/adql-cookbook).

### Functions
The following functions (among others) are available when using ADQL:
- `(A)SIN`, `(A)COS`, `(A)TAN`, (using radians)
- `DEGREES`, `RADIANS`
- `EXP`, `LOG`, `LOG10`, `POWER`, `SQRT`
- `ABS`, `RAND`, `PI`, `MOD`
- `ROUND`, `CEILING`, `FLOOR`, `TRUNCATE`

### Geometries
It is possible to create geometries and then compare them!
- Geometries: `BOX`, `CIRCLE`,  `POINT`, `POLYGON`

The first argument for these _'Constructors'_ is the Coordinate System. So for standard equatorial coordinates, (equinox J2000.0) use `'ICRS'`:

The signature of point is `POINT(coordsys, longcoord, latcoord)` (coordinates both in __degrees__). A point on the North pole is given by:

```
POINT('ICRS', 0, 90)
```

And a circle of 10 degrees around this point is then given by:

```
CIRCLE('ICRS', 0, 90, 10)
```

We can use these geometries in so called 'Predicate' functions: `INTERSECT` and `CONTAINS`.

- `INTERSECT(geometry1, geometry2)` takes two geometries and results in `1` if they intersect, and 0 otherwise.

    So from our example: 
    
    ```INTERSECT(POINT('ICRS', 0, 90), CIRCLE('ICRS', 0, 90, 10)) = 1```
    
    and 
    
    ```INTERSECT(POINT('ICRS', 0, 0),  CIRCLE('ICRS', 0, 90, 10)) = 0```

- `CONTAINS(geometry1, geometry2)` again takes two geometries, and results in `1` if geomerty 1 is __fully__ contained by geometry 2:

    Again:
    
    ```CONTAINS(POINT('ICRS', 0, 90), CIRCLE('ICRS', 0, 90, 10)) = 1```
    
    ```CONTAINS(POINT('ICRS', 0, 0),  CIRCLE('ICRS', 0, 90, 10)) = 0```

### Back to queries
The most common usage of these geometries and predicate functions, is to select all data entries in a specific region of the sky.

Here we use some imaginary table `observations` which contains (among others) `ra` and `dec` coordinates of the observations:

```
SELECT * FROM observations
WHERE CONTAINS(POINT('ICRS', ra, dec), CIRCLE('ICRS', 0, 90, 10)) = 1
```

Or, using the `BOX` geometry:

```
SELECT * FROM observations
WHERE CONTAINS(POINT('ICRS', ra, dec), BOX('ICRS', 0, 0, 10, 1)) = 1
```

The signature of `BOX` is `BOX(coordsys, longcoord, latcoord, longExtent, latExtent)`. The _'coords'_ are the center of the rectangle, and the _'extents'_ give the (total) length of the sides.

### Controlling Output
ADQL offers some additional functionality for selecting the `TOP X` rows of your query:

```
SELECT TOP 100 *
FROM observations
```

# Tap Interfaces

- Topcat [link](http://www.star.bris.ac.uk/~mbt/topcat/#docs)
- TAPHandle [link](http://saada.unistra.fr/taphandle/#)
- Python's: [Astroquery](https://astroquery.readthedocs.io/en/latest/), [PyVO](https://pyvo.readthedocs.io/en/latest/)

Example query to use:
```
SELECT * FROM observations.raw
WHERE CONTAINS(POINT('ICRS', ra, dec), CIRCLE('ICRS', 200, 30, 20)) = 1
```

Cicle of: $ra=13h20m0s, dec=30^\circ, radius = 20^\circ$

# Python Interface & Example Queries

See other notebook TAPExamples !

# Future Development
- Automated data reduction 
- Expose reduced data as another table!
- Small system bugs
- Any other improvements we can think of!